##### Commencing the data extraction process... 

Our aim is to uncover the unique respiratory care values generated by individual hospitals from the eICU database. This vital information provides us with the foundational knowledge needed for developing a standardized reporting metric for ventilator values across healthcare institutions.

# Step 1: Loading the data... 

We are delving into three separate datasets: 

# hospital data, patient data, and respiratory charting data

##### for a complete listing of the columns available, please see

https://eicu-crd.mit.edu/eicutables/hospital/

https://eicu-crd.mit.edu/eicutables/patient/

https://eicu-crd.mit.edu/eicutables/respiratorycharting/

Each one is a piece of the puzzle that, when combined, gives us a clearer picture of the real-world ventilation practices.

In [ ]:
# call forth the pandas
import pandas as pd

# Load the csv files into pandas dataframes
print("Loading data...")
hospital_df = pd.read_csv('/Users/Paul/Downloads/hospital.csv')
patient_df = pd.read_csv('/Users/Paul/Downloads/patient.csv')
respiratorycharting_df = pd.read_csv('/Users/Paul/Downloads/respiratorycharting.csv', dtype={6: str})
print("Data loaded successfully.")

# Step 2: Merging the patient data with the respiratory charting data...

# This step involves cross-referencing and merging the patient and respiratory 
# charting datasets based on the patientunitstayid. Our focus is on the hospitalid 
# and respchartvaluelabel. This combined information will provide the link between 
# a specific patient's stay in a hospital and the unique respiratory care value, 
# a crucial part of understanding the individualized ventilator care.

# Merge patient and respiratorycharting dataframes to link hospitalid and respchartvaluelabel
print("Merging data...")
intermediate_df = pd.merge(patient_df[['patientunitstayid', 'hospitalid']], respiratorycharting_df[['patientunitstayid', 'respchartvaluelabel']], on='patientunitstayid', how='inner')

# Step 3: Removing duplicate entries... 

# To ensure the integrity of our analysis, we take the step of eliminating 
# any duplicate data. This process helps us to maintain the uniqueness of each 
# hospital's respiratory care value.

# Drop duplicates to ensure each respchartvaluelabel is unique per hospital
print("Removing duplicates...")
intermediate_df = intermediate_df.drop_duplicates()
print("Duplicates removed successfully.")

# Step 4: Linking hospital data... 

# The final step in our data processing is merging the combined patient and 
# respiratory charting data with the hospital data. This holistic approach brings 
# together the specific ventilator values and the hospitals generating these values, 
# forming a comprehensive dataset for labeling purposes.

# Merge the intermediate dataframe with hospital dataframe
print("Merging with hospital data...")
final_df = pd.merge(intermediate_df, hospital_df, on='hospitalid', how='inner')
print("Data merged successfully.")

# Step 5: Saving the processed data... 

# With the final dataframe assembled, we commit it to a CSV file. 
# This file encapsulates the harmonized data, representing a truthful account of 
# ventilation practices across the hospitals. This is a vital resource for our research 
# goal - developing a standardized reporting metric for ventilator values.

# Save the final dataframe to a new csv file
print("Saving data to CSV...")
final_df.to_csv('/Users/Paul/Downloads/newfile.csv', index=False)
print("Data saved successfully. Process completed.")


# Completion of the process... 

Our data journey has successfully culminated. We are now equipped with a refined dataset, ready to further explore and establish the truth about standard reporting metrics in ventilator values. The foundation for our research has been laid.

## But first...

We must survey the land and understand how it is being used in daily practice...

# Commencing the data analysis process... 

We have successfully created a meticulously curated dataset containing respiratory care value labels across various hospitals. Our next step is to delve into this dataset and unveil insights related to hospital-specific respiratory care practices.

In [2]:
import pandas as pd

# Step 1: Displaying Dataframe Information... 
    
# The first step in our exploration is to examine the fundamental structure of the dataset. 
# We are presented an overview of the dataframe, including the number of entries, the presence of 
# any missing values, and the data type of each column. This step helps us to familiarize 
# ourselves with the dataset and understand its composition.

# Load the csv file into a pandas dataframe
print("Loading data...")
df = pd.read_csv('/Users/Paul/Downloads/respcarevaluelabelbyhospital.csv')
print("Data loaded successfully.")

# Step 2: Calculating Descriptive Statistics for Numeric Columns... 

# We then delve into the numeric columns of our dataset, offering a statistical summary. 
# This summary includes aspects such as the mean, standard deviation, minimum and maximum 
# values, and quartiles. It provides a quick glance at the spread and central tendency of 
# the numeric data, helping us gauge the distribution and variability.

# Generate descriptive statistics
print("\nDataframe information:")
print(df.info())

print("\nDescriptive statistics for numeric columns:")
print(df.describe())

# Set option to display all rows
pd.set_option('display.max_rows', None)

# Step 3: Exploring Non-Numeric Columns... 

# Our exploration doesn't stop at the numeric data. We extend it to the non-numeric 
# columns such as 'respchartvaluelabel', 'numbedscategory', 'teachingstatus', and 'region'. 
# For each of these, we count the unique values and their frequencies, providing a clear 
# picture of the data distribution.

# For non-numeric columns, we can look at the count of unique values
for col in ['respchartvaluelabel', 'numbedscategory', 'teachingstatus', 'region']:
    print(f"\nCounts of unique values for {col}:")
    print(df[col].value_counts())


Loading data...
Data loaded successfully.

Dataframe information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 734352 entries, 0 to 734351
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   patientunitstayid    734352 non-null  int64 
 1   hospitalid           734352 non-null  int64 
 2   respchartvaluelabel  734352 non-null  object
 3   numbedscategory      644205 non-null  object
 4   teachingstatus       734352 non-null  object
 5   region               695409 non-null  object
dtypes: int64(2), object(4)
memory usage: 33.6+ MB
None

Descriptive statistics for numeric columns:
       patientunitstayid     hospitalid
count       7.343520e+05  734352.000000
mean        1.865162e+06     285.232252
std         1.046993e+06     121.048580
min         1.411680e+05      56.000000
25%         8.807570e+05     171.000000
50%         1.754087e+06     273.000000
75%         3.019526e+06     416.000000
max

f    508509
t    225843
Name: teachingstatus, dtype: int64

Counts of unique values for region:
Midwest      281122
South        191046
West         154388
Northeast     68853
Name: region, dtype: int64


After examining the data, we can see the problem with the data.

# There is too many unique values that are reporting the same metric, inadvertently losing measurements and making analysis difficult due to messy data.

So, where is this data coming from and how can we begin to correct this?

In [3]:
# Our first detailed exploration of the Hospitals and Regions that contributed to this
# can be determined by counting the amount of times that each label was used in the data 
# at each hospital vs. region.

# Count of unique hospital IDs
unique_hospital_ids = df['hospitalid'].nunique()
print(f"Total unique hospital IDs: {unique_hospital_ids}")

# Count of unique patient IDs
unique_patient_ids = df['patientunitstayid'].nunique()
print(f"Total unique patient IDs: {unique_patient_ids}")

# Count of unique hospital IDs per region
unique_hospital_ids_per_region = df.groupby('region')['hospitalid'].nunique()
print("\nTotal unique hospital IDs per region:")
print(unique_hospital_ids_per_region)

# Count of unique respchartvaluelabel per region
unique_respchartvaluelabel_per_region = df.groupby('region')['respchartvaluelabel'].nunique()
print("\nTotal unique respchartvaluelabel per region:")
print(unique_respchartvaluelabel_per_region)



Total unique hospital IDs: 185
Total unique patient IDs: 124598

Total unique hospital IDs per region:
region
Midwest      64
Northeast    13
South        44
West         41
Name: hospitalid, dtype: int64

Total unique respchartvaluelabel per region:
region
Midwest      125
Northeast     35
South         82
West          35
Name: respchartvaluelabel, dtype: int64


In [8]:
# Sort the dataframe by unique patient IDs and unique respiratory care labels in descending order
combined_df = combined_df.sort_values(by=['Unique Patient IDs', 'Unique Resp Care Labels'], ascending=[False, False])

print(combined_df)


            Unique Resp Care Labels  Unique Patient IDs
hospitalid                                             
167                              18                5727
73                               14                5127
176                              19                4026
199                              10                3929
252                              23                3323
443                              16                3039
411                               9                2919
165                              19                2854
148                              20                2716
264                              20                2639
171                              18                2607
122                              12                2563
281                              13                2495
413                               9                2321
458                              57                2275
338                              21             